In [1]:
#3
#athena db config - lifearchitectmodels.csv
#AAI-540 Group 3 FP

In [2]:
import boto3
import sagemaker
from pyathena import connect
import pandas as pd
sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
ingest_create_athena_db_passed = False

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [3]:
database_name = "dsoaws"
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [4]:
statement = "CREATE DATABASE IF NOT EXISTS {}".format(database_name)
print(statement)
pd.read_sql(statement, conn)

CREATE DATABASE IF NOT EXISTS dsoaws


/tmp/ipykernel_7198/2532319348.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement, conn)


""


In [5]:
statement = "SHOW DATABASES"
df_show = pd.read_sql(statement, conn)
df_show.head(5)

/tmp/ipykernel_7198/1677879827.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_show = pd.read_sql(statement, conn)


,database_name
0,default
1,dsoaws
2,sagemaker_featurestore


In [6]:
s3_data_path = 's3://sagemaker-us-east-1-883727203134/lifearchitectmodels.csv'
s3_table_path = 's3://sagemaker-us-east-1-883727203134/table2/'

In [7]:
!aws s3 cp {s3_data_path} {s3_table_path}

copy: s3://sagemaker-us-east-1-883727203134/lifearchitectmodels.csv to s3://sagemaker-us-east-1-883727203134/table2/lifearchitectmodels.csv


In [8]:
#table2 - lifearchitectmodels.csv
table_name_csv = 'lifearchitectmodels'
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)
ingest_create_athena_table_csv_passed = False
dataexplore = pd.read_csv('lifearchitectmodels.csv')
dataexplore.head()

,(745) Permalink:,https://lifearchitect.ai/models-table,Unnamed: 2,Unnamed: 3,Timeline view:,https://lifearchitect.ai/timeline,Unnamed: 6,Unnamed: 7,Unnamed: 8,The Memo:,https://lifearchitect.ai/memo,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18
0,Model,Lab,Playground,Parameters \n(B),Tokens \ntrained (B),Ratio Tokens:Params\n(Chinchilla scaling≥20:1),"ALScore \n""ALScore"" is a quick and dirty ratin...",MMLU,MMLU\n-Pro,GPQA,HLE,Training dataset,Announced\n▼,Public?,Paper / Repo,Arch,Tags,Notes,Count (rough)
1,AuroraGPT (ScienceGPT),Argonne National Laboratory,https://lifearchitect.ai/auroragpt/,2000,30000,15:1,25.8,NaN,NaN,NaN,NaN,NaN,TBA,🔴,NaN,NaN,NaN,Three models targeted in Jul/2024: AuroraGPT-7...,l
2,Avocado,Meta AI,https://www.cnbc.com/2025/12/09/meta-avocado-a...,2000,30000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TBA,NaN,NaN,NaN,NaN,Due 2026.,q
3,DeepSeek-R2,DeepSeek-AI,https://www.reuters.com/technology/artificial-...,1200,15640,14:1,14.4,NaN,NaN,NaN,NaN,NaN,TBA,🟢,https://docs.google.com/document/d/e/2PACX-1vT...,MoE,"Reasoning, SOTA","Hybrid MoE, 1.2TA78B. 5.2PB corpus=1.3Qa token...",k
4,GPT-6,OpenAI,https://lifearchitect.ai/gpt-6/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TBA,NaN,NaN,NaN,SOTA,Due 2026.,f


In [9]:
dataexplore.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 754 entries, 0 to 753
Data columns (total 19 columns):
 #   Column                                 Non-Null Count  Dtype 
---  ------                                 --------------  ----- 
 0   (745) Permalink:                       754 non-null    object
 1   https://lifearchitect.ai/models-table  753 non-null    object
 2   Unnamed: 2                             634 non-null    object
 3   Unnamed: 3                             735 non-null    object
 4   Timeline view:                         661 non-null    object
 5   https://lifearchitect.ai/timeline      658 non-null    object
 6   Unnamed: 6                             658 non-null    object
 7   Unnamed: 7                             249 non-null    object
 8   Unnamed: 8                             145 non-null    object
 9   The Memo:                              192 non-null    object
 10  https://lifearchitect.ai/memo          53 non-null     object
 11  Unnamed: 11        

In [10]:
!aws s3 cp s3://sagemaker-us-east-1-883727203134/lifearchitectmodels.csv \
           s3://sagemaker-us-east-1-883727203134/table2/

copy: s3://sagemaker-us-east-1-883727203134/lifearchitectmodels.csv to s3://sagemaker-us-east-1-883727203134/table2/lifearchitectmodels.csv


In [11]:
!aws s3 ls s3://sagemaker-us-east-1-883727203134/table2/

2026-01-27 03:23:15     247859 lifearchitectmodels.csv


In [31]:
#sql table creation
statement = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
         Question STRING,
         Response STRING,
         Overall STRING,
         Age_18_34 STRING,
         Age_35_49 STRING,
         Age_50_64 STRING,
         Age_65_up STRING,
         Gender_Women STRING,
         Gender_Men STRING,
         College_No_College_College STRING,
         College_No_College_No_college STRING,
         Income_three_way_Lower_Income STRING,
         Income_three_way_Middle_Income STRING,
         Income_three_way_Upper_Income STRING,
         Generation_Gen_Z STRING,
         Generation_Millenial STRING,
         Generation_Gen_X STRING,
         Generation_Boomer STRING,
         Generation_Silent STRING,
         Race_Native_American STRING,
         Race_Asian STRING,
         Race_Hispanic STRING,
         Race_Black STRING,
         Race_White STRING,
         Occupation_Category_Office_Job STRING,
         Occupation_Category_Customer_Facing_Service_Job STRING,
         Occupation_Category_Manual_Not_Customer_Facing STRING,
         Occupation_Category_Education_Medicine STRING,
         Occupation_Category_Building_Trades STRING
) ROW FORMAT DELIMITED 
  FIELDS TERMINATED BY ',' 
  LINES TERMINATED BY '\\n' 
LOCATION '{}'
TBLPROPERTIES ('skip.header.line.count'='1')""".format(
    database_name, table_name_csv, s3_table_path
)

In [32]:
pd.read_sql(statement, conn)

/tmp/ipykernel_4059/618921652.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement, conn)


""


In [33]:
statement = "SHOW TABLES in {}".format(database_name)
df_show = pd.read_sql(statement, conn)
df_show.head(5)

/tmp/ipykernel_4059/84333453.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_show = pd.read_sql(statement, conn)


,tab_name
0,aimodelpoll
1,dataset_csv2
2,llmdatasetcsv


In [34]:
statement = """SELECT * FROM {}.{} LIMIT 10""".format(
    database_name, table_name_csv
)
print(statement)
df = pd.read_sql(statement, conn)
df.head()

SELECT * FROM dsoaws.aimodelpoll LIMIT 10


/tmp/ipykernel_4059/888292236.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(statement, conn)


,question,response,overall,age_18_34,age_35_49,age_50_64,age_65_up,gender_women,gender_men,college_no_college_college,...,race_native_american,race_asian,race_hispanic,race_black,race_white,occupation_category_office_job,occupation_category_customer_facing_service_job,occupation_category_manual_not_customer_facing,occupation_category_education_medicine,occupation_category_building_trades
0,AI Services Used,ChatGPT,34.8%,48.8%,40.7%,32.8%,17.1%,34.1%,35.5%,41.4%,...,33.0%,40.2%,37.5%,35.2%,33.7%,39.9%,34.2%,29.9%,36.0%,31.7%
1,AI Services Used,Claude,1.7%,3.1%,1.8%,1.2%,0.6%,1.6%,1.8%,1.7%,...,1.9%,2.2%,2.4%,2.4%,1.4%,2.2%,2.3%,1.9%,2.2%,2.0%
2,AI Services Used,Gemini,24.3%,27.9%,29.9%,23.8%,16.0%,22.5%,26.3%,25.4%,...,25.5%,28.0%,27.6%,30.4%,22.1%,28.6%,27.9%,28.1%,27.6%,28.5%
3,AI Services Used,Grok,3.8%,6.3%,4.3%,2.9%,1.8%,2.8%,4.9%,3.5%,...,3.8%,4.3%,4.7%,3.9%,3.6%,4.3%,4.0%,4.4%,3.8%,4.8%
4,AI Services Used,Copilot,12.2%,11.5%,14.0%,14.5%,9.2%,10.8%,13.7%,16.6%,...,11.2%,15.2%,11.5%,12.9%,11.9%,14.9%,10.2%,10.9%,13.4%,12.1%
